### 라이브러리 설치
[Adaptive Experimentation Platform](https://ax.dev/)

In [1]:
!pip install ax-platform

### Reproducibility
[PyTorch](https://pytorch.org/docs/stable/notes/randomness.html)

In [2]:
import os
import random
import numpy as np
import torch

In [3]:
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
# try:
#     torch.use_deterministic_algorithms(True)
# except AttributeError:
#     torch.set_deterministic(True)
# torch.backends.cudnn.deterministic = True

### MNIST 데이터
[MNIST](http://yann.lecun.com/exdb/mnist/)  
[PyTorch](https://pytorch.org/vision/stable/datasets.html#torchvision.datasets.MNIST)

In [4]:
import torchvision.transforms as transforms

from torchvision.datasets import MNIST

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

trainset = MNIST('./data', train=True, download=True, transform=transform)
testset = MNIST('./data', train=False, transform=transform)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### PyTorch 분류기

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
import torch.nn as nn

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(28 * 28, 10)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        out = self.fc(x)
        return out

### Bayesian Optimization with Ax-Platform

In [9]:
def train(model, loader, params, device):
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(model.parameters(), lr=params.get("lr", 0.001))

    model.train()

    for _ in range(5):
        for xs, ys in loader:
            xs, ys = xs.to(device), ys.to(device)
            output = model(xs)
            loss = criterion(output, ys)
            optim.zero_grad()
            loss.backward()
            optim.step()

    return model

In [10]:
def evaluate(model, loader, params, device):
    correct, total = 0, 0

    model.to(device)
    
    model.eval()

    with torch.no_grad():
        for xs, ys in loader:
            xs, ys = xs.to(device), ys.to(device)
            output = model(xs)

            _, predicted = torch.max(output, 1)

            total += ys.size(0)
            correct += (predicted == ys).sum().item()

    return correct / total

In [11]:
from torch.utils.data import DataLoader

In [12]:
def eval_fn(params):
    trainloader = DataLoader(trainset, batch_size=params.get("batchsize", 1024), shuffle=True)
    testloader = DataLoader(testset, batch_size=params.get("batchsize", 1024))
    model = Net()
    trained_model = train(model, trainloader, params, device)
    acc = evaluate(trained_model, testloader, params, device)
    return acc

In [13]:
from ax.service.managed_loop import optimize

In [14]:
best_params, _, _, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [0.001, 0.1], "log_scale": True},
        {"name": "batchsize", "type": "range", "bounds": [512, 600]},
        {"name": "epochs", "type": "range", "bounds": [10, 30]}
    ]    # 하이퍼파라미터와 범위
    evaluation_function=eval_fn,    # 학습과 테스트 함수
    objective_name='accuracy',    # 목표 (eval_fn이 반환하는 값)
    total_trials=20,    # 트라이얼 수
    random_seed=SEED
)

[INFO 07-11 19:45:08] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-11 19:45:08] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter batchsize. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-11 19:45:08] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter epochs. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-11 19:45:08] ax.modelbridge.dispatch_utils: Using GPEI (Bayesian optimization) since there are more continuous parameters than there are categories for the unordered categorical parameters.
[INFO 07-11 19:45:08] ax.modelbridge.dispatch_utils: 

#### Optimized Hyperparameters 출력

In [15]:
print(best_params)

{'lr': 0.004281206840537801, 'batchsize': 587, 'epochs': 20}
